# Notes #
1. Data preparation
    + Removed 30% instances due to missing values.  
    Is it significant?


**Rough notes**
1. 

---

**Data preparation**

In [7]:
# Load dataset info #
import automation_script
import pandas as pd
import numpy as np
from os import path

dataset_name = "UCI Mushroom"
dataset_info = automation_script.get_url(dataset_name)

In [2]:
# Data preparation #

names = ['classes', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment',
        'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
        'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring',
        'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color',
        'population', 'habitat']
url = "../data/mushroom.data.csv" if path.exists("../data/dataset.csv.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=",", header=None, names=names, index_col=False)

# Check for columns that contain missing values #

print("Fields with missing values")
col_names = data.columns
num_data = data.shape[0]
for c in col_names:
    num_non = data[c].isin(["?"]).sum()
    if num_non > 0:
        print (c)
        print (num_non)
        print ("{0:.2f}%".format(float(num_non) / num_data * 100))
        print ("\n")

data = data[data["stalk-root"] != "?"]

# Convert categorical fields #

for col in names:
    b, c = np.unique(data[col], return_inverse=True)
    data[col] = c

# Split the dataset into test and train datasets #
feature_list = names[1:23]
X = data.loc[:, feature_list]
Y = data[['classes']]

data.head()


Fields with missing values
stalk-root
2480
30.53%




,classes,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,2,...,2,5,5,0,0,1,3,1,3,5
1,0,5,2,7,1,0,1,0,0,2,...,2,5,5,0,0,1,3,2,2,1
2,0,0,2,6,1,3,1,0,0,3,...,2,5,5,0,0,1,3,2,2,3
3,1,5,3,6,1,6,1,0,1,3,...,2,5,5,0,0,1,3,1,3,5
4,0,5,2,3,0,5,1,1,0,2,...,2,5,5,0,0,1,0,2,0,1


---

**Keras Model**

In [3]:
config = {
    'epoch': 2500,
    'batch_size': 100,
    'verbose': 0,
    'model_info': {
        'loss':'binary_crossentropy',
        'optimizer':'adam',
        'metrics':['accuracy']
    }
}

keras_score,keras_params = automation_script.get_keras_params(X,Y,dataset_info,config)

Using TensorFlow backend.


3387/3387 [==============================] - 0s 51us/step

acc: 96.72%


---

**Scikit model**

In [4]:
scikit_score, scikit_params = automation_script.get_scikit_params(X,Y)

0.9521700620017715


C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


---

**Kfold validation**

In [5]:
config = {
    'epoch': 500,
    'batch_size': 100,
    'splits':10,
    'model_info': {
        'loss':'binary_crossentropy',
        'optimizer':'adam',
        'metrics':['accuracy']
    }
}

kfold_acc = automation_script.get_kfold(X,Y,config)

acc: 93.81%
acc: 96.11%
acc: 95.75%
acc: 96.99%
acc: 96.81%
acc: 94.51%
acc: 96.81%
acc: 96.45%
acc: 96.45%
acc: 96.80%
96.05% (+/- 1.02%)

   -------------   



<Figure size 640x480 with 1 Axes>

[[2096   36]
 [  79 1176]]

   -------------   

             precision    recall  f1-score   support

    Class 1       0.96      0.98      0.97      2132
    Class 2       0.97      0.94      0.95      1255

avg / total       0.97      0.97      0.97      3387



---

#### Write back to Master sheet ####

In [6]:
dataset_info['scikit_params'] = scikit_params
dataset_info['keras_params'] = keras_params
dataset_info['type'] = 'Binary'
accuracy_values = {
    'keras': keras_score,
    'scikit': scikit_score,
    'kfold': kfold_acc
}

automation_script.write_to_mastersheet(dataset_info,X,Y,accuracy_values)

---

# Pending #
1. 